### BINOMIAL PRICING MODEL FOR SPY CALL OPTION
#### FE-620 FINAL PROJECT

IMPORTS

In [73]:
import yfinance as yf
import pandas as pd
import numpy as np

READ & PULL DATA

In [74]:
spyopt = pd.read_excel('spy_opt_data.xlsx')


IDENTIFY HISTORICAL VOLATILITY

In [75]:
spy = yf.download('SPY', start='2023-01-01', end='2023-11-71')
spy = spy['Adj Close']
spy.head()

[*********************100%%**********************]  1 of 1 completed


1 Failed download:


['SPY']: ValueError('unconverted data remains: 1')


Series([], Name: Adj Close, dtype: float64)

In [76]:
def binom_call(s0 : float, k : float, n : float, t : float, r : float, sig : float):
    # calc tree params
    dt = t/n
    u = np.exp(sig*np.sqrt(dt))
    d = 1/u
    p = (np.exp(r*dt) - d)/(u - d)
    q = 1 - p
    # initialize tree
    c = np.zeros((n+1, n+1))
    # calc option prices at maturity
    for i in range(n+1):
        c[n, i] = max(s0*u**i*d**(n-i)-k, 0)
    # backpropagate
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            c[i, j] = np.exp(-r*dt)*(p*c[i+1, j+1] + q*c[i+1, j])
            # check exercise
            c[i, j] = max(c[i, j], s0*u**j*d**(i-j)-k)
    # return price and df
    return c[0, 0], pd.DataFrame(c)

INITIALIZE MODEL PARAMETERS

In [77]:
s0 = 421.82 # market price as of 6/1/23
k = 455 # call strike
n = 122 # biz days to expiration (6/1/23 - 11/17/23)
t = 122/252 # tmt per annum
r = 5.27638/100 # 3M SOFR as of 6/1/23
sig = 0

In [78]:
price, pricedf = binom_call(s0=100, k=101, n=3, t=1, r=.01, sig=0.2)
print("price: ", price)
print("pricedf: ", pricedf)

price:  8.629081997392712
pricedf:            0          1          2          3
0  8.629082   0.000000   0.000000   0.000000
1  2.632779  15.040594   0.000000   0.000000
2  0.000000   5.439915  25.314485   0.000000
3  0.000000   0.000000  11.240090  40.398246
